In [ ]:
# !pip install rlcard[torch]

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import rlcard
from rlcard.agents import RandomAgent
from rlcard.games.base import Card
import torch
from IPython.display import clear_output
from rlcard.utils import reorganize, print_card


In [2]:
env = rlcard.make(
    'limit-holdem-collaborative',
    config={
        'seed': 42,
        'game_num_players': 3
    }
)
# agent1, agent2, agent3 = RandomAgent(num_actions=4), RandomAgent(num_actions=4), RandomAgent(num_actions=4)
agent1 = torch.load('./checkpoint_collab/collab_brain2_more_info.pth')
agent2 = torch.load('./checkpoint_collab/collab_brain2_more_info.pth')
agent3 = torch.load('./checkpoint_collab/comp_brain1.pth')
env.set_agents([agent1, agent2, agent3])
agents = env.agents



In [3]:
game = env.game.init_game()

old_hands = [env.game.players[i].hand for i in range(env.num_players)]

cards = ['C7', 'H2', 'SA', 'HA', 'D8', 'S2']

env.game.players[0].hand = [Card('C', '7'), Card('H', '2')]
env.game.players[1].hand = [Card('S', 'A'), Card('H', 'A')]
env.game.players[2].hand = [Card('D', '8'), Card('S', '2')]

for h in old_hands:
  if str(h[0]) not in cards:
    env.game.dealer.deck.append(h[0])
  if str(h[0]) not in cards:
    env.game.dealer.deck.append(h[1])

trajectories = [[] for _ in range(env.num_players)]

curr_player_id = env.get_player_id()
curr_state = env.get_state(curr_player_id)

trajectories[curr_player_id].append(curr_state)

while not env.is_over():
  curr_player = agents[curr_player_id]
  curr_action = curr_player.eval_step(curr_state)
  
  trajectories[curr_player_id].append(curr_action)

  curr_state, curr_player_id = env.step(curr_action[0], curr_player.use_raw)


  if not env.game.is_over():
    trajectories[curr_player_id].append(curr_state)

for player_id in range(env.num_players):
  state = env.get_state(player_id)
  trajectories[player_id].append(state)


# Payoffs
payoffs = env.get_payoffs()

trajectories = reorganize(trajectories, payoffs)

In [4]:
trajectories[0][0][0]['obs'][0:53]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1.])

In [5]:
def visualize_trajectory(traj, traj_env, player_id):
  p_traj = traj[player_id]

  round2name = {0: 'Pre Flop', 1: 'Flop', 2: 'Turn', 3: 'River'}
  all_actions = ['call', 'raise', 'fold', 'check']
  round_count = 0
  pub_cards = []

  print(f"Player {player_id} cards")
  print_card(p_traj[0][0]['raw_obs']['hand'])
  print("-"*50)
  print(round2name[round_count])
  print("-"*50)

  for t in p_traj:
    s, a, r, ns, done = t
    old_len = len(pub_cards)

    pub_cards = s['raw_obs']['public_cards']

    if old_len != len(pub_cards):
      round_count += 1
      print("-"*50)
      print(f"{round2name[round_count]}")
      print("-"*50)
      print_card(pub_cards) 

    print(all_actions[a[0]])

    if done:
      print("-"*50)
      print(f"Player {player_id} got {r}")



In [38]:
visualize_trajectory(trajectories, env, 2)

Player 2 cards
┌─────────┐   ┌─────────┐
│8        │   │2        │
│         │   │         │
│         │   │         │
│    ♦    │   │    ♠    │
│         │   │         │
│         │   │         │
│        8│   │        2│
└─────────┘   └─────────┘
--------------------------------------------------
Pre Flop
--------------------------------------------------
fold
--------------------------------------------------
Player 2 got -1.0


In [12]:
import pickle

In [13]:
with open('traj.pkl', 'wb') as f:
    pickle.dump(trajectories, f)

In [22]:
print(trajectories[0][0])

[{'legal_actions': OrderedDict([(0, None), (1, None), (2, None)]), 'obs': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0.]), 'raw_obs': {'hand': ['C7', 'H2'], 'public_cards': [], 'all_chips': [1, 2, 2], 'my_chips': 1, 'legal_actions': ['call', 'raise', 'fold'], 'raise_nums': [0, 2, 2, 4], 'raise_by_collaborator': [1, 0, 0, 0]}, 'raw_legal_actions': ['call', 'raise', 'fold'], 'action_record': [(2, 'call'), (0, 'call'), (1, 'check'), (2, 'raise'), (0, 'raise'), (1, 'call'), (2, 'call'), (0, 'raise'), (1, 'call'), (2, 'raise'), (0, 'call'), (1, 'call'), (2, 'raise'), (0, 'raise'), (1, 'raise'), (2, 'raise'), (0, 'call'), (1, 'call')]}, (0, {'probs': {'call': 0.7371799101758778, 'raise': 0.259436079158322